# Reproducible Figures 2–4 (Clean Notebook)

This notebook reproduces Figures 2–4 from the scoping review using the minimal required inputs located in `data/`.

**Inputs (required):**
- `ML_Stunting_Standardized_28_BESTPRACTICE_WITH_SCORES_FINAL_v2.csv`

**Inputs (required for Figure 3 only):**
- `pipeline_components_28studies.csv` with columns: `Author_Year`, `Preprocessing`, `Feature_selection`, `Hyperparameter_tuning`, `Class_imbalance_handling`, `Model_Family` (0/1 or Yes/No)

**Outputs:** PNG + PDF saved to `outputs/figures/` at 600 dpi.


In [ ]:
# ---- Setup ----
dir.create("outputs/figures", recursive = TRUE, showWarnings = FALSE)

# Packages (minimal set)
pkgs <- c("tidyverse","janitor","scales","patchwork","colorspace","circlize","ComplexHeatmap")
to_install <- pkgs[!pkgs %in% installed.packages()[,"Package"]]
if (length(to_install)) install.packages(to_install)

if (!requireNamespace("ComplexHeatmap", quietly = TRUE)) {
  if (!requireNamespace("BiocManager", quietly = TRUE)) install.packages("BiocManager")
  BiocManager::install("ComplexHeatmap", ask = FALSE, update = FALSE)
}

suppressPackageStartupMessages({
  library(tidyverse)
  library(janitor)
  library(scales)
  library(patchwork)
  library(colorspace)
  library(circlize)
  library(ComplexHeatmap)
})

# Save helpers
save_gg <- function(p, stem, w, h, dpi = 600) {
  ggsave(file.path("outputs/figures", paste0(stem, ".png")), p, width = w, height = h, dpi = dpi)
  ggsave(file.path("outputs/figures", paste0(stem, ".pdf")), p, width = w, height = h, device = cairo_pdf)
}

save_ht <- function(ht, stem, w, h, dpi = 600) {
  pdf(file.path("outputs/figures", paste0(stem, ".pdf")), width = w, height = h, useDingbats = FALSE)
  draw(ht, heatmap_legend_side = "bottom", annotation_legend_side = "bottom")
  dev.off()
  png(file.path("outputs/figures", paste0(stem, ".png")), width = w*dpi, height = h*dpi, res = dpi)
  draw(ht, heatmap_legend_side = "bottom", annotation_legend_side = "bottom")
  dev.off()
}

writeLines(capture.output(sessionInfo()), "outputs/session_info.txt")


In [ ]:
# ---- Load data ----
std <- readr::read_csv("data/ML_Stunting_Standardized_28_BESTPRACTICE_WITH_SCORES_FINAL_v2.csv", show_col_types = FALSE) %>%
  janitor::clean_names()

# Required columns check
req <- c("author_year","fusion_level_updated","model_family","validation_type_std",
         "external_temporal_validation","independent_test_evaluation","calibration_assessment",
         "subgroup_auditing","interpretability_reporting","code_availability","data_availability","ethics_governance",
         "bestpractice_score_8")
missing <- setdiff(req, names(std))
if (length(missing)) stop(paste("Missing required columns in standardized dataset:", paste(missing, collapse = ", ")))


## Figure 2 — Modality and Fusion strategy

In [ ]:
# ---- Figure 2 ----
std2 <- std %>%
  mutate(
    modality = if_else(is.na(fusion_level_updated), "Unimodal", "Multimodal"),
    fusion = fusion_level_updated
  )

# Modality panel
mod_df <- std2 %>%
  count(modality) %>%
  mutate(p = n/sum(n))

p_modality <- ggplot(mod_df, aes(x = 1, y = p, fill = modality)) +
  geom_col(width = 0.6, color = NA) +
  coord_flip() +
  scale_y_continuous(labels = percent_format(accuracy = 1), expand = c(0,0)) +
  geom_text(aes(label = paste0(n, " (", percent(p, accuracy = 1), ")")),
            position = position_stack(vjust = 0.5), size = 6) +
  scale_fill_manual(values = c("Multimodal"="#0072B2","Unimodal"="#D9D9D9")) +
  labs(title = "Modality", x = NULL, y = "Studies (%)") +
  theme_minimal(base_size = 16) +
  theme(panel.grid.minor = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        legend.position = "bottom",
        plot.title = element_text(face="bold"))

# Fusion panel (multimodal only)
fus_df <- std2 %>%
  filter(modality == "Multimodal") %>%
  count(fusion) %>%
  mutate(p = n/sum(n)) %>%
  mutate(fusion = factor(fusion, levels = c("Two-stage","Late","Early")))

p_fusion <- ggplot(fus_df, aes(x = 1, y = p, fill = fusion)) +
  geom_col(width = 0.6, color = NA) +
  coord_flip() +
  scale_y_continuous(labels = percent_format(accuracy = 1), expand = c(0,0)) +
  geom_text(aes(label = paste0(n, " (", percent(p, accuracy = 1), ")")),
            position = position_stack(vjust = 0.5), size = 6) +
  scale_fill_manual(values = c("Early"="#F8766D","Late"="#00BA38","Two-stage"="#619CFF")) +
  labs(title = "Fusion strategy (multimodal only)", x = NULL, y = "Multimodal studies (%)", fill = "Fusion strategy") +
  theme_minimal(base_size = 16) +
  theme(panel.grid.minor = element_blank(),
        axis.text.y = element_blank(),
        axis.ticks.y = element_blank(),
        legend.position = "bottom",
        plot.title = element_text(face="bold"))

fig2 <- p_modality / p_fusion + plot_layout(heights = c(1, 1.1))
fig2
save_gg(fig2, "Figure2_Modality_Fusion", w = 7.2, h = 3.6, dpi = 600)


## Figure 3 — Pipeline components reporting heatmap

This figure requires a small, dedicated input file (`data/pipeline_components_28studies.csv`) because the four pipeline-component indicators are not present in the standardized dataset you provided.

In [ ]:
# ---- Figure 3 ----
pipe_path <- "data/pipeline_components_28studies.csv"
if (!file.exists(pipe_path)) {
  stop(paste0("Missing required file for Figure 3: ", pipe_path,
              "\nCreate it with columns: Author_Year, Preprocessing, Feature_selection, Hyperparameter_tuning, Class_imbalance_handling, Model_Family"))
}

pipe <- readr::read_csv(pipe_path, show_col_types = FALSE) %>%
  janitor::clean_names()

reqp <- c("author_year","preprocessing","feature_selection","hyperparameter_tuning","class_imbalance_handling","model_family")
missingp <- setdiff(reqp, names(pipe))
if (length(missingp)) stop(paste("Missing required columns in pipeline file:", paste(missingp, collapse = ", ")))

# Normalize to 0/1
to01 <- function(x) {
  if (is.numeric(x)) as.integer(x > 0)
  else as.integer(tolower(trimws(as.character(x))) %in% c("1","yes","y","true","reported"))
}

pipe01 <- pipe %>%
  mutate(across(c(preprocessing,feature_selection,hyperparameter_tuning,class_imbalance_handling), to01)) %>%
  mutate(completeness = preprocessing + feature_selection + hyperparameter_tuning + class_imbalance_handling) %>%
  arrange(desc(completeness), author_year)

mat <- pipe01 %>%
  select(preprocessing,feature_selection,hyperparameter_tuning,class_imbalance_handling) %>%
  as.matrix()
rownames(mat) <- pipe01$author_year
colnames(mat) <- c("Preprocessing","Feature
selection","Hyperparameter
tuning","Class imbalance
handling")

# Color mapping
col_fun <- c("0"="#D9D9D9","1"="#0072B2")

# Row annotation: completeness bars + model family strip
ra <- rowAnnotation(
  `Completeness
(0–4)` = anno_barplot(pipe01$completeness, gp = gpar(fill="grey60", col=NA), border = FALSE,
                                      axis_param = list(at = 0:4, labels = 0:4)),
  `Model family` = pipe01$model_family,
  col = list(`Model family` = structure(
    hcl.colors(length(unique(pipe01$model_family)), "Dark 3"),
    names = unique(pipe01$model_family)
  ))
)

# Top annotation: % reported per column
pct <- colMeans(mat) * 100
ta <- HeatmapAnnotation(
  ` ` = anno_barplot(pct, gp = gpar(fill="grey60", col=NA), border = FALSE,
                     axis_param = list(at = c(0,20,40,60,80), labels = c(0,20,40,60,80))),
  annotation_name_side = "left"
)

ht_pipe <- Heatmap(
  mat,
  name = "Reported",
  col = col_fun,
  cluster_rows = FALSE,
  cluster_columns = FALSE,
  show_row_names = FALSE,
  show_column_names = TRUE,
  top_annotation = ta,
  right_annotation = ra,
  column_title = NULL,
  heatmap_legend_param = list(at = c(0,1), labels = c("Not reported/unclear","Reported"))
)

ht_pipe
save_ht(ht_pipe, "Figure3_Pipeline_Components", w = 10.5, h = 7.0, dpi = 600)


## Figure 4 — Evidence atlas (validation + reporting & ethics)

In [ ]:
# ---- Figure 4 ----
atlas <- std %>%
  transmute(
    author_year,
    model_family,
    validation = validation_type_std,
    External_Temporal_Validation = as.integer(external_temporal_validation > 0),
    Independent_Test_set = as.integer(independent_test_evaluation > 0),
    Calibration_Assessment = as.integer(calibration_assessment > 0),
    Fairness_Subgroup_audit = as.integer(subgroup_auditing > 0),
    Explainability_reported = as.integer(interpretability_reporting > 0),
    Code_available = as.integer(code_availability > 0),
    Data_available = as.integer(data_availability > 0),
    Ethics_governance = as.integer(ethics_governance > 0),
    readiness = bestpractice_score_8
  ) %>%
  mutate(
    readiness_grp = case_when(
      readiness <= 2 ~ "Low (0–2)",
      readiness <= 5 ~ "Moderate (3–5)",
      TRUE ~ "High (6–8)"
    ),
    readiness_grp = factor(readiness_grp, levels = c("Low (0–2)","Moderate (3–5)","High (6–8)"))
  ) %>%
  arrange(readiness_grp, desc(readiness), author_year)

mat_val <- atlas %>%
  select(External_Temporal_Validation, Independent_Test_set, Calibration_Assessment) %>%
  as.matrix()
mat_rep <- atlas %>%
  select(Fairness_Subgroup_audit, Explainability_reported, Code_available, Data_available, Ethics_governance) %>%
  as.matrix()

rownames(mat_val) <- atlas$author_year
rownames(mat_rep) <- atlas$author_year

colnames(mat_val) <- c("External/temporal
validation","Independent
test set","Calibration
assessment")
colnames(mat_rep) <- c("Fairness/
subgroup audit","Explainability
reported","Code
available","Data
available","Ethics/
governance")

col_fun <- c("0"="#D9D9D9","1"="#0072B2")

# Side annotations: model family and validation type
fam_levels <- unique(atlas$model_family)
val_levels <- unique(atlas$validation)

fam_cols <- structure(hcl.colors(length(fam_levels), "Dark 3"), names = fam_levels)
val_cols <- structure(hcl.colors(length(val_levels), "Set 2"), names = val_levels)

ra <- rowAnnotation(
  `Model family` = atlas$model_family,
  Validation = atlas$validation,
  col = list(`Model family` = fam_cols, Validation = val_cols),
  annotation_legend_param = list(
    `Model family` = list(title = "Model family"),
    Validation = list(title = "Validation")
  )
)

# Left readiness barplot
la <- rowAnnotation(
  `Readiness
(0–8)` = anno_barplot(atlas$readiness, gp = gpar(fill="grey60", col=NA), border = FALSE,
                                   axis_param = list(at = c(0,2,4,6,8), labels = c(0,2,4,6,8))),
  annotation_name_side = "bottom",
  width = unit(1.3, "cm")
)

# Top % reported bars
ta_val <- HeatmapAnnotation(
  ` ` = anno_barplot(colMeans(mat_val)*100, gp=gpar(fill="grey60", col=NA), border=FALSE,
                     axis_param=list(at=c(0,20,40,60,80), labels=c(0,20,40,60,80))),
  annotation_name_side="left"
)
ta_rep <- HeatmapAnnotation(
  ` ` = anno_barplot(colMeans(mat_rep)*100, gp=gpar(fill="grey60", col=NA), border=FALSE,
                     axis_param=list(at=c(0,20,40,60,80), labels=c(0,20,40,60,80))),
  annotation_name_side="left"
)

ht_val <- Heatmap(mat_val, name="Reported", col=col_fun, cluster_rows=FALSE, cluster_columns=FALSE,
                  show_row_names=FALSE, show_column_names=TRUE, top_annotation=ta_val,
                  row_title="Low (0–2)", column_title="Validation",
                  heatmap_legend_param=list(at=c(0,1), labels=c("Not reported/unclear","Reported")))

ht_rep <- Heatmap(mat_rep, name="Reported", col=col_fun, cluster_rows=FALSE, cluster_columns=FALSE,
                  show_row_names=FALSE, show_column_names=TRUE, top_annotation=ta_rep,
                  column_title="Reporting & Ethics",
                  heatmap_legend_param=list(at=c(0,1), labels=c("Not reported/unclear","Reported")))

# Split rows by readiness group: draw as three blocks using row_split
ht <- (ht_val + ht_rep) %>%
  add_heatmap(ra, side="right")  # placeholder to keep object type

# ComplexHeatmap doesn't use patchwork; create a combined HeatmapList and draw with annotations
ht_list <- ht_val + ht_rep
ht_list <- ht_list + ra
ht_list <- la + ht_list

# Apply row split
ht_list <- ht_list
# draw uses row_split argument inside Heatmap; easier: rebuild each with row_split
ht_val2 <- Heatmap(mat_val, name="Reported", col=col_fun, cluster_rows=FALSE, cluster_columns=FALSE,
                   show_row_names=FALSE, show_column_names=TRUE, top_annotation=ta_val,
                   row_split = atlas$readiness_grp,
                   column_title="Validation",
                   heatmap_legend_param=list(at=c(0,1), labels=c("Not reported/unclear","Reported")))
ht_rep2 <- Heatmap(mat_rep, name="Reported", col=col_fun, cluster_rows=FALSE, cluster_columns=FALSE,
                   show_row_names=FALSE, show_column_names=TRUE, top_annotation=ta_rep,
                   row_split = atlas$readiness_grp,
                   column_title="Reporting & Ethics",
                   heatmap_legend_param=list(at=c(0,1), labels=c("Not reported/unclear","Reported")))

ht_atlas <- la + (ht_val2 + ht_rep2) + ra
ht_atlas
save_ht(ht_atlas, "Figure4_Evidence_Atlas", w = 12.5, h = 8.0, dpi = 600)
